In [1]:
import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

## 加载训练集文件和测试集文件

In [2]:
data_text,data=[],[]
stopWords=[]

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv('/home/user10000411/dataset/图书分类文本数据集/train_clean.csv', sep='\t'),
        pd.read_csv('/home/user10000411/dataset/图书分类文本数据集/dev_clean.csv', sep='\t'),
        pd.read_csv('/home/user10000411/dataset/图书分类文本数据集/test_clean.csv', sep='\t')
        ])
    print("读取数据集完成")
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
    stopWords = open('stopwords.txt').readlines()
    print("取停用词完成")
    return data, data_text,datatext, stopWords


In [3]:
data, data_text, datatext, stopWords=load_data()

load Pre_process
读取数据集完成
取停用词完成


In [7]:
def trainer_tfidf():
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    count_vect = TfidfVectorizer(stop_words=stopWords, max_df=0.6, ngram_range=(1,2))
    tfidf = count_vect.fit(data_text)
    print('train tfidf_embedding')
    #返回是一个稀疏矩阵
    return tfidf


In [8]:
tfidf = trainer_tfidf()

train tfidf_embedding


In [9]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000) 
        
    w2v.build_vocab(datatext)

    w2v.train(datatext,
              total_examples=w2v.corpus_count,
              epochs=15,
              report_delay=1)
    return w2v

In [10]:
w2v = trainer_w2v()

train word2vec Embedding


In [11]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    # 根据数据集训练FastTExt的词向量
    # size：生成的向量维度，
    # window: 移动窗口，
    # aphla: 学习率，
    # min_count: 对低于该频率的词进行截断
    print('train fast_embedding')
    fast = models.FastText(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000)
    fast.build_vocab(datatext)

    fast.train(datatext,
              total_examples=fast.corpus_count,
              epochs=15,
              report_delay=1)
    
    return fast

In [12]:
fast= trainer_fasttext()

train fast_embedding


In [13]:
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    joblib.dump(tfidf,'/home/user10000411/notespace/Embedding/models/tfidf_model')
    print('save tfidf model')    

    w2v.save('/home/user10000411/notespace/Embedding/models/w2v_model_50000')
    print('save word2vec model')

    fast.save('/home/user10000411/notespace/Embedding/models/fast_model')
    print('save fast model')

    
saver()
    

save tfidf model
save word2vec model
save fast model


In [14]:
def load():
    '''
    函数说明：加载训练好的模型
    '''
    tfidf =  joblib.load('/home/user10000411/notespace/Embedding/models/tfidf_model')
    print('load tfidf_embedding model')
    w2v = models.KeyedVectors.load('/home/user10000411/notespace/Embedding/models/w2v_model_50000')
    print('load w2v_embedding model')
    fast = models.KeyedVectors.load('/home/user10000411/notespace/Embedding/models/fast_model')
    print('load fast_embedding model')
load()

load tfidf_embedding model
load w2v_embedding model
load fast_embedding model
